In [47]:
import os
import time
import datetime
from PyPDF2 import PdfReader
import google.generativeai as genai
from google.generativeai import caching

In [48]:
from dotenv import load_dotenv
# Load variables from the .env file
load_dotenv()

api_key = os.getenv("GENAI_API_KEY")
if not api_key:
    raise EnvironmentError("GENAI_API_KEY environment variable is not set.")

genai.configure(api_key=api_key)

In [49]:
pdf_dir = 'GEMINI-TEST-Folder/'
extracted_texts = dict()
for file in os.listdir(pdf_dir):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_dir, file)
        reader = PdfReader(pdf_path)
        text = ''.join(page.extract_text() for page in reader.pages)
        extracted_texts[file] = text
print("Text extracted from all PDFs.")

# import pdf2image
# import pytesseract

# pdf_dir = 'GEMINI-TEST-Folder/'
# extracted_texts = dict()

# for file in os.listdir(pdf_dir):
#     if file.endswith('.pdf'):
#         pdf_path = os.path.join(pdf_dir, file)
#         images = pdf2image.convert_from_path(pdf_path)
#         text = ''
#         for image in images:
#             text += pytesseract.image_to_string(image)
#         extracted_texts[file] = text

# print("Text extracted from all PDFs.")

Text extracted from all PDFs.


In [50]:
print(extracted_texts['1.pdf']) #1.pdf is not being processed correctly (needs OCR): use smallpdf.com to convert 1.pdf into 1o.pdf (which has searchable text)
#and rerun previous cell

EX-10 4 ex-10.htm
 
Exhibit 10
 
 
  
 
 
  
 
 
  
 
  
 
 
  
 
 
 
 


In [51]:
print(extracted_texts['1o.pdf'])

GUARANTY
OF
ACCOUNT
Inconsidemtion oftheopening, and/orthecontinuing, byRBCCapitalMarkets,
LLC
(“RBC
CM”)
oftheaccountoraccountsidentified inScheduleA
hereto(whichseparately or
jointly,withanyandallrenewalsthereofandasmayberemembered byRBCCMfiomtimeto
time,arecollectively hereinafter referredtoasthe“Guaranteed Account”) witli,orotherwise
giving
creditintheGuaranteed AccounttotheownerorownersoftheGuaranteed Account
(hereinafter
referred
toasthe“Customer”), theundersigned (hereinafter referred toasthe
“Guarantor”),
on
the
terms
and
subject
to
the
conditions
hereinafter
setforth,hereby
unconditionally
agrees
topaytoRBCCM,ondemand, anyindebtedness whichmaynowor
hereafier
beowingtoRBCCMbytheCustomer, including, withoutlimitation, anyintereston
any
such
indebtedness,
together
with
any
reasonable
costsandexpenses ofenforcing this
Guaranty
(collectively,
the“Liabilities”).
1.
Guuanrv.
TheGuarantor unconditionally andirrevocably guarantees toRBCCM
thepunctualpaymentandperformance oftheLiabiliti

In [52]:
print(extracted_texts['4.pdf'])

EX-10 3 10.2earnoutagreement.htm 10.2
Execution Version
EARNOUT  AGREEMENT
 
THIS EARNOUT AGREEMENT (this “Agreement ”) is made  as of the September 10,
2024, by and among Farmhouse Inc., a Nevada Corporation (“Buyer ”), Nappy Dranks, LLC a
Delaware limited liability company ("Nappy Boy "), and GSB Holdings, LLC, a Nevada limited
liability company ("GSB " and, together with Nappy Boy, “Sellers ”), and Thrown, LLC, a
Delaware limited liability company (the “Company ”).  Com pany, Buyer and Sellers are
collectively referred to herein as the “ Parties ”. 
 
WITNESSETH:
 
WHEREAS, Compa ny, Sellers and Buyer are parties to that certain equity exchange
agreement dated as of the same date hereof (the “Share Exchange Agreement ”) which provides
for Buyer  to purchase all of the Membership Units of the Company (terms used in this
Agreement which are not defined herein shall have the meaning given to such term in the Share
Exchange Agreement);
 
WHEREAS, the Share Exchange Agreement provides th

In [53]:
def split_text_into_chunks(text, max_tokens=3000):
    sentences = text.split(". ")
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_tokens:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

def create_prompt(file, total):
    return f"""
    The above text in {file} discussed legal principles. I provided you with several sections of {file}, which you have stored in your memory. Consider all of the sections together: chunks 1 - {total}. I will not provide the previous chunks again. Please refer to them in your analysis.
    Does the combined text support or contradict the principle of 'contra proferentem', a legal doctrine favoring interpretation against the drafter in ambiguous contract terms?

    - If any chunk contradicts the principle of 'contra proferentem', the overall response should be "NO."
    - If all chunks either support or are neutral, the response should be "YES."

    Reference your cached content in your response. Respond only with "YES" or "NO".
    """
def create_prompt1(file, chunk, i, total):
    return f"""
    This is chunk {i} of {total} in {file}. Analyze this text to see if it supports or contradicts the principle of 'contra proferentem', a legal doctrine favoring interpretation against the drafter in ambiguous contract terms.
    Store your analysis of this chunk but do not respond. Once I provide the last chunk, synthesize your analysis across all chunks to answer the question above.
    - If this chunk contradicts the principle, store this note.
    - If this chunk supports the principle, store this note.
    - If this chunk neither supports nor contradicts, note it as "neutral."
    When the final chunk is provided, synthesize your analysis:
    - Respond "NO" if any chunk contradicts the principle, even if others support it.
    - Respond "YES" only if all chunks either support or are neutral.
    Here is the text for chunk {i}:
    
    ----------- START OF CHUNK {i} -------------

    {chunk}

    ------------ END OF CHUNK {i} --------------
    """


In [54]:
def contra_proferentem_analysis(fileName):
  results = dict()
  if fileName not in extracted_texts:
    print(f"File not found: {fileName}")
    return
  text = extracted_texts[fileName]

  print(f"Processing file: {fileName}")
  chunks = split_text_into_chunks(text)
  file_results = []
  threshold = 200000
  text = extracted_texts.get(fileName, "").ljust(threshold, 'Z')
  cache = caching.CachedContent.create(
      model='models/gemini-1.5-flash-001',
      display_name=f'contra proferentem analysis {fileName}', # used to identify the cache
      system_instruction=(
          'You are an expert law analyzer, and your job is to answer '
          'the user\'s query based on the law file that you have access to.'
      ),
      contents= text,
      ttl=datetime.timedelta(minutes=10),
  )
  print(f'Cache created for {fileName}, {cache}')
  model = genai.GenerativeModel.from_cached_content(cached_content=cache)
  for i, chunk in enumerate(chunks):
      print(f"Processing chunk {i+1}/{len(chunks)}.")
      try:
        prompt = create_prompt1(fileName, chunk, i+1, len(chunks))
        response = model.generate_content(prompt).text
        print(response)
        time.sleep(2)
      except Exception as e:
        print(f"Error processing chunk {i+1}/{len(chunks)}: {e}")
        time.sleep(60)
        prompt = create_prompt1(fileName, chunk, i, len(chunks)-1)
        response = model.generate_content(prompt).text
        print(response)
        time.sleep(2)
  prompt = create_prompt(fileName, len(chunks))
  response = model.generate_content(prompt).text
  print(response)
  results[fileName] = response

In [55]:
contra_proferentem_analysis("1o.pdf")

Processing file: 1o.pdf
Cache created for 1o.pdf, CachedContent(
    name='cachedContents/e5zfh3c4bbst',
    model='models/gemini-1.5-flash-001',
    display_name='contra proferentem analysis 1o.pdf',
    usage_metadata={
        'total_token_count': 50980,
    },
    create_time=2024-12-27 03:15:29.512303+00:00,
    update_time=2024-12-27 03:15:29.512303+00:00,
    expire_time=2024-12-27 03:25:28.771979+00:00
)
Processing chunk 1/6.
```json
{
  "chunk_1": "supports"
}
```
Processing chunk 2/6.
- This chunk supports the principle of contra proferentem. 

Processing chunk 3/6.
- **Neutral:** This chunk primarily defines events of default and remedies.  While it references the "Account Documents," it does not provide specific language or situations that would clearly favor or disfavor the drafter in interpretation. 

Processing chunk 4/6.
- **Chunk 4: Neutral** 

Processing chunk 5/6.
- **Neutral.** This chunk doesn't explicitly favor either side in terms of ambiguity. It largely outline

In [56]:
contra_proferentem_analysis("4.pdf")

Processing file: 4.pdf
Cache created for 4.pdf, CachedContent(
    name='cachedContents/bkigrvdbb3x5',
    model='models/gemini-1.5-flash-001',
    display_name='contra proferentem analysis 4.pdf',
    usage_metadata={
        'total_token_count': 49793,
    },
    create_time=2024-12-27 03:16:04.522751+00:00,
    update_time=2024-12-27 03:16:04.522751+00:00,
    expire_time=2024-12-27 03:26:03.802314+00:00
)
Processing chunk 1/7.
**Chunk 1 Analysis:** Neutral 

This chunk primarily introduces the parties, the agreement's purpose, and the relationship to a prior agreement. It does not explicitly address any contract terms or ambiguity. Therefore, it is neither supportive nor contradictory of the contra proferentem doctrine. 

Processing chunk 2/7.
This chunk is neutral. 

It primarily defines terms and sets forth the terms of the earnout payment. It does not directly involve any ambiguous terms that might be subject to interpretation, nor does it explicitly favor or disfavor the drafte